Training and forecasting the solar power of Hokkaido Island in Japna
Training 2017, Forecast 2018

https://stackoverflow.com/questions/35139108/how-to-install-xgboost-in-anaconda-python-windows-platform

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import csv 
import matplotlib.pyplot as plt
import pytz
import xgboost as xgb

In [2]:
import os
cwd = os.getcwd()
cwd
# os.chdir('pvinhokkaidodatadistribution')
# C:\Users\Mhdella\Google Drive\Jupyter Folder\PV_Hokkaido_forecast_contest\
os.chdir('C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution')
nwd=os.getcwd()
nwd

'C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution'

In [3]:
# file_name='Merg_Ens_Agg_42_44_mes17_18.csv'
# file_name='Merg_Ens_Agg_43_mes17_18.csv'

# file_name='Merg_Ens_Agg_42_44_mes16_17_18.csv'
file_name='Merg_Ens_mmaxmin_Agg_42_44_mes16_18.csv'

# file_name='Merg_Res_Comb_S1_S2_2018_tz_Jp.csv'
# file_mane='Merg_Res_Comb_S1_S2_2017_tz_Jp.csv'

df_hok=pd.read_csv(file_name) 
df_hok.head()
# df_hok.tail()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [4]:
#### This cell just to check out the dataset with different coordination lat 43 and lat42-44 from ECMWF
df_hok1=pd.read_csv('Merg_Ens_Agg_42_44_mes17_18.csv') 
df_hok2=pd.read_csv('Merg_Ens_Agg_43_mes17_18.csv')

df_mns= {'col1': df_hok1.mean(), 'col2': df_hok2.mean()}
# df_mns

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

In [5]:
import sklearn 
import numpy as np
from sklearn.metrics import mean_squared_error
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
df_hok.head()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [6]:
print(list(df_hok))

['DATE', 'TIME', 'S1[MW]', 'S2[MW]', 'solar[kW/m2]', 'temp[deg C]', 'solar[kW/m2].1', 'temp[deg C].1', 'solar[kW/m2].2', 'temp[deg C].2', 'solar[kW/m2].3', 'temp[deg C].3', 'u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']


In [7]:
#### df_feats=df_hok[df_hok['ssrdc'] != 0] ## dropping night hours, when clear-sky solar irradiance (SSRDC) forecasts=0, and this can be applied in real forecasts  

#### df_feats=df_hok.set_index('DATE')

############################# Not dropping night hours

df_tar_feats=df_hok.set_index(pd.to_datetime(df_hok['DATE'] + ' ' + df_hok['TIME']))

############## In case of forecast solar power  #######

Targ=df_tar_feats['S1[MW]']+df_tar_feats['S2[MW]']
Targ = Targ.replace(np.nan, 0, regex=True)

df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
             'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
             'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]
############### In case of forecast solar irradiance  #######

# Targ=df_tar_feats['ssrd']  #ssrd is the mean of ssrd data from all ensemble members
# df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
#              'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
#              'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]


df_tar_feats.insert(loc=0, column='Targ', value=Targ)

df_feats.head()
df_feats.tail()

print(len(list(df_feats)), len(list(df_tar_feats)))
# Targ.tail()


36 49


In [8]:
len(df_feats)
# len(Targ)
print(len(Targ), max(Targ), Targ.mean())

52603 312.6 37.385689029143066


In [9]:
data_hr=df_tar_feats  #this without measurements features

print(len(list(data_hr)))

49


In [50]:
# ind_date_time= pd.date_range('2019-03-08 00:00','2019-03-09 00:00',freq='120T')
ind_date_time= pd.date_range('2019-03-08 04:00','2019-03-09 03:30',freq='30T')

Hours= ind_date_time.strftime('%H:%M')
Hours
# for num in range(0,len(Hours),6):
#     print(Hours[num])

04:00
07:00
10:00
13:00
16:00
19:00
22:00
01:00


In [52]:
##### R=dfddf ### Just to stop to check out before run forecast models

ind_date_time= pd.date_range('2019-03-08 00:00','2019-03-08 23:30',freq='30T')
# ind_date_time= pd.date_range('2019-03-08 04:00','2019-03-09 03:30',freq='180T')


Hours= ind_date_time.strftime('%H:%M')

for num in range(0,len(Hours),2):
    hr=Hours[num][:-3]
    hr_nx=Hours[num+1][:-3]
    df_hr=df_tar_feats.between_time(Hours[num],Hours[num+1])
    print(Hours[num], 'to', Hours[num+1], len(df_hr), max(df_hr['Targ']), df_hr['Targ'].mean(), df_hr['Targ'].std())
    

00:00 to 00:30 2192 0.0 0.0 0.0
01:00 to 01:30 2192 0.0 0.0 0.0
02:00 to 02:30 2192 0.0 0.0 0.0
03:00 to 03:30 2192 0.0 0.0 0.0
04:00 to 04:30 2192 11.9 0.30305656934306585 1.2397216897715433
05:00 to 05:30 2192 56.400000000000006 4.2052463503649635 9.305438054841794
06:00 to 06:30 2191 138.7 19.0628023733455 29.50632033828794
07:00 to 07:30 2190 220.60000000000002 47.74899543378996 57.90973084733816
08:00 to 08:30 2190 287.5 82.67557077625571 85.36399499440849
09:00 to 09:30 2192 308.2 109.35205291970803 104.40599979690991
10:00 to 10:30 2192 312.0 122.41706204379562 111.74838562406212
11:00 to 11:30 2192 312.6 125.3797445255477 112.51787524840006
12:00 to 12:30 2192 312.4 121.10780109489042 109.74505969362221
13:00 to 13:30 2192 306.79999999999995 108.93713503649634 101.33405961100473
14:00 to 14:30 2192 281.79999999999995 82.8536040145987 83.52415194298344
15:00 to 15:30 2192 216.39999999999998 48.115921532846706 58.14388358014166
16:00 to 16:30 2192 132.0 19.988686131386842 30.6101

In [31]:
####Select features of the data 
Targ=df_hr['Targ']
df_feats_w_mes=df_hr.drop('Targ', axis=1) #Features including measurements and weather forecasts
df_feats_no_mes =df_hr.drop(df_tar_feats.columns[range(0,13)], axis=1)  #Features of weather forecasts only
print(len(df_feats_no_mes), len(df_feats_w_mes), len(Targ))
# print(len(list(df_feats_no_mes)), len(list(df_feats_w_mes)), len(list(Targ)))

# df_feats=df_feats_w_mes  ## the measurements lenght is less than the weather since there are not exist in forecast period
df_feats=df_feats_no_mes   
df_feats.describe()
df_feats.head()

2192 2192 2192


,u10,v10,t2m,asn,sd,ssrdc,ssrd,tcc,tciw,tclw,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
2016-01-01 23:00:00,3.760489,0.552673,-3.069997,0.828247,0.015845,0.0,0.0,0.687062,0.014540,0.017966,...,-14.874915,0.598475,7.450000e-09,0.0,0.0,0.018576,0.000111,0.0,0.0,3.641058
2016-01-01 23:30:00,3.780177,0.737312,-2.966578,0.828181,0.015851,0.0,0.0,0.700182,0.015622,0.017497,...,-14.778769,0.598309,7.450000e-09,0.0,0.0,0.023220,0.000135,0.0,0.0,3.332800
2016-01-02 23:00:00,6.002045,-0.403519,0.332011,0.828186,0.017302,0.0,0.0,0.424855,0.016948,0.009181,...,-9.295264,0.595360,7.450000e-09,0.0,0.0,0.000000,0.000000,0.0,0.0,5.920933
2016-01-02 23:30:00,6.053597,-0.490884,0.280019,0.828106,0.017321,0.0,0.0,0.423766,0.016141,0.008575,...,-9.287878,0.595201,7.450000e-09,0.0,0.0,0.000000,0.000000,0.0,0.0,5.907053
2016-01-03 23:00:00,5.209865,-0.171494,-2.269745,0.825329,0.017712,0.0,0.0,0.579794,0.012634,0.004803,...,-11.138536,0.585427,7.450000e-09,0.0,0.0,0.078083,0.000243,0.0,0.0,3.768079


In [32]:
X_train=df_feats.loc['1/1/2016':'12/31/2016']
X_test=df_feats.loc['1/1/2017':'12/31/2017']

# X_train=df_feats.loc['1/1/2017':'12/31/2017']
# X_test=df_feats.loc['1/1/2018':'12/31/2018']

Y_train=Targ.loc['1/1/2016':'12/31/2016']
Y_test=Targ.loc['1/1/2017':'12/31/2017']

# Y_train=Targ.loc['1/1/2017':'12/31/2017']
# Y_test=Targ.loc['1/1/2018':'12/31/2018']

X_train.head()
# X_train.tail()
# len(X_train)

# Y_train.head()
# len(Y_train)
Y_train.tail()

2016-12-29 23:30:00    0.0
2016-12-30 23:00:00    0.0
2016-12-30 23:30:00    0.0
2016-12-31 23:00:00    0.0
2016-12-31 23:30:00    0.0
Name: Targ, dtype: float64

In [33]:
len(df_hr)
hr

'23'

In [17]:
from Hokk_Hourly_Forecast_Models import model_intrv0, model_intrv1, model_intrv2, model_intrv3, model_intrv4

if int(hr)<4:
    print('Yes4')
    model_intrv0(Y_train, X_train, Y_test, X_test, Targ)
elif int(hr)>=4 and int(hr_nx)<=7:
    print('Yes7')
    model_intrv1(Y_train, X_train, Y_test, X_test, Targ)
elif int(hr)>=8 and int(hr_nx)<=11:
    print('Yes11')
    model_intrv2(Y_train, X_train, Y_test, X_test, Targ)
elif int(hr)>=12 and int(hr_nx)<=15:
    print('Yes15')
    model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
elif int(hr)>=16 and int(hr_nx)<=19:
    print('Yes19')
    model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
else:
    print('No')


No


In [20]:
# Y_test
int(hr)
int(hr_nx)
# hr=8
# if hr>=5 and hr<=7:
#     print('Yes')
# else:print('NO')


12

In [ ]:
Y_test